In [21]:
import pandas as pd
import numpy as np
import pendulum
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [22]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [23]:
# Environment Variables
load_dotenv()

api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

In [24]:
# clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [25]:
# Parameters
end = pendulum.datetime(2024,7,4)
start = end.subtract(days=2)

In [26]:
# Get all current assets
asset_request = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    asset_class = AssetClass.US_EQUITY,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets] # Convert objects to dicts

In [27]:
df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['fractionable'] & df['shortable']]

df.to_csv("assets_test.csv",index=False)

df.head(1)

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
434,4a0d2ab4-8091-4b80-8d25-7ba2ff417116,AssetClass.US_EQUITY,AssetExchange.NASDAQ,CASS,"Cass Information Systems, Inc Common Stock",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"


In [28]:
symbols = df['symbol'].to_list()

In [29]:
# Get previous days market data
bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

bars = data_client.get_stock_bars(bars_request)

In [30]:
df = bars.df

df = df.reset_index()

df.head(3)

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AKRO,2024-07-02 04:00:00+00:00,23.03,23.2100,22.23,22.38,432928.0,8713.0,22.447360
1,AKRO,2024-07-03 04:00:00+00:00,22.22,22.3900,21.74,21.93,216345.0,4390.0,21.978922
2,AMZA,2024-07-02 04:00:00+00:00,41.27,41.8376,41.27,41.48,28067.0,421.0,41.512591
